# Tutorial: center of the small Quantum group

This page appears both rendered to HTML in the docs, and as an interactive notebook in the `/examples` folder in the repository. 

## Introduction

This tutorial will show how to compute the center of the small quantum group, as described in the two papers by Annach Lachowska and You Qi:

https://arxiv.org/abs/1604.07380v3

https://arxiv.org/abs/1703.02457v3

Let $G$ be a complex simple Lie group, and let $P$ be a parabolic subgroup. Then we consider the cotangent bundle of the associated partial flag variety:
$$
    \tilde{\mathcal N}_P :=T^*(G/P)
$$

We are then interested in computing
$$HH^s(\tilde{\mathcal N}_P)\cong \bigoplus_{i+j+k=s}H^i(\tilde{\mathcal N}_P,\wedge^jT\tilde{\mathcal N}_P)^k$$

This can be computed by using the BGG resolution. We define the following module:
$$M_j^k = \bigoplus_r \operatorname{Sym}^{j-r+k/2}\mathfrak u_P\otimes \wedge^r\mathfrak g\otimes \wedge^{j-r}\mathfrak n_P$$

Let $\Delta\colon\mathfrak p\to \mathfrak g\oplus \mathfrak u_P\otimes\mathfrak n_P$ be given by the inclusion in the first component and in the second component by the adjoint action (after identifying $\operatorname{End}(\mathfrak n_P)$ with $\mathfrak u_P\otimes \mathfrak n_P$). Then $\Delta$ induces a map $M_{j-1}^k\to M_j^k$. We define the module

$$E_j^k = M_j^k\big/\Delta(M_{j-1}^k)$$

Then the cohomology of the BGG resolution of $E_j^k$ in degree $i$ with respect to a dominant weight $\mu$ computes the multiplicity of $\mu$ of $H^i(\tilde{\mathcal N}_P,\wedge^jT\tilde{\mathcal N}_P)^k$.

## Example

We begin by loading the required packages.

In [1]:
from bggcohomology.la_modules import BGGCohomology
from bggcohomology.bggcomplex import BGGComplex
from bggcohomology.quantum_center import *
from bggcohomology.weight_set import WeightSet
from bggcohomology.cohomology import compute_diff

Now let us compute a simple example and set $G=A_2$. We will compute $H^0(\tilde{\mathcal N}_0,\wedge^2T\tilde{\mathcal N}_0)^{-2}$. Here the subscript $0$ indicates the principal block. The principal block corresponds to the parameter `subset=[]`. The other parameters are `s,i,j,k`, and they satisfy `i+j+k=s` and `j>=i`.

If we want to change the parabolic subalgebra $P$ then we can change `subset` to e.g. `subset=[1]` for 
$$\mathfrak p = \mathfrak b\oplus\mathbb C\langle e_1\rangle=\mathbb C\langle e_1,f_1,f_2,f_{12} \rangle,\qquad \mathfrak u_P = \mathbb C\langle e_{12},e_2\rangle,\qquad \mathfrak n_P = \mathbb C\langle f_{12},f_2\rangle$$

In [14]:
import datetime
td = datetime.timedelta(seconds=float(1000.123))
str(td)

'0:16:40.123000'

In [2]:
BGG = BGGComplex('A2')
s = 1
i = 1
j = 2
k = s-i-j
print('i+j=%d, j-i=%d, k=%d\n'%(i+j,j-i,k))

subset = [1]
mjk = Mjk(BGG,j,k,subset=subset)
cohom = BGGCohomology(BGG, mjk, coker=Eijk_basis(BGG,j,k,subset=subset, sparse=True))
cohom.cohomology_LaTeX(i=i,only_non_zero=False,print_modules=True,print_betti=True,complex_string=r'\tilde{\mathcal N}_0,\wedge^2T\tilde{\mathcal N}_0)^{-2}')

i+j=3, j-i=1, k=-2



<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Bigraded tables
The program can also produce bigraded tables. We decompose impose a bigrading on the total cohomology $HH^s(\tilde{\mathcal N}_P)$ by $a=i+j$ and $b = j-i$. Since the computation can take a while a progress bar is provided. 

Due to the $\mathfrak{sl}_2$ symmetry in the bigraded table, we only actually need to compute half of it. We can choose to either compute the whole table, or only half if it through the `half_only` parameter, and extend by symmetry. It's default value is `False`. 

The bigraded table can be rendered in LaTeX through the `display_ab_dic` function, which takes an additional parameter `extend_half` which computes the second half of the table from the first half through symmetry considerations. The produced LaTeX code can easily be copied. The code below also saves the tables to a LaTeX document. 

The parameter `compact` determines how the weight decomposition of the cohomology is displayed. E.g. with `compact=False` it would display $\mathbb C\oplus L(\alpha_1+\alpha_2)^2$ and with `compact=True` it gives $\mathbb C L_{1,1}^2$.

In [3]:
%%prun -D prun.log
import pickle
import os
import itertools
from tqdm.auto import tqdm
os.makedirs('pickles', exist_ok=True)
os.makedirs('tables', exist_ok=True)

# Only compute cohomology for particular highest weight module
mu = (0,0,0,0)
#mu=(0,0)

# the parameters we actually want to change
diagram = 'A4'
BGG = BGGComplex(diagram)
subset=[]

# compute only half of the table, extend by symmetry
half_only = True
extend_half = half_only

# Exclude the top-left to bottom-right diagonal. If s=0, these should all be the trivial rep.
exclude_diagonal = True

# Display in full form
compact = True

# Load results if already computed
load_pickle = False

s=0
#for method in [0,1]:
#for s in itertools.count():
method = 0
for s in [0]:
    picklefile = os.path.join('pickles',f'{diagram}-s{s}-{subset}.pkl')
    if load_pickle and os.path.isfile(picklefile):
        previous_cohom = pickle.load(open(picklefile, 'rb'))
    else:
        previous_cohom = None
    texfile = os.path.join('tables',f'{diagram}-s{s}-{subset}.tex')
    cohom_dic = dict()
    with tqdm(all_abijk(BGG,s=s,subset=subset,half_only=half_only)) as inner_pbar:
        with tqdm(leave=None) as outer_pbar:
            map_pbar = tqdm()
            for a,b,i,j,k in inner_pbar:
                if previous_cohom is not None and (a,b) in previous_cohom:
                    cohom_dic[(a,b)]=previous_cohom[(a,b)]
                    inner_pbar.update()
                    continue
                if exclude_diagonal and s==0 and (a==b):
                    cohom_dic[(a,b)]=[((0,)*BGG.rank,1)]
                    inner_pbar.update()
                    continue
                inner_pbar.set_description('i+j= %d, j-i = %d'%(a,b))
                #coker = Eijk_basis(BGG,j,k,subset=subset,pbar=outer_pbar)
                mjk = Mjk(BGG,j,k,subset=subset)
                outer_pbar.set_description('Initializing cohomology')
                coker=Eijk_basis(BGG,j,k,subset=subset,sparse=True)
                cohom = BGGCohomology(BGG, mjk, 
                                      coker=coker,pbars = [outer_pbar,map_pbar])
                outer_pbar.set_description('Computing cohomology')
                cohom_list = cohom.cohomology(i, mu=mu)
                cohom_dic[(a,b)] = cohom_list
                with open(picklefile, 'wb') as f:
                    pickle.dump(cohom_dic,f)   
    print('-'*50)
    print(f'type {diagram}, table for s={s}:')
    cohom = BGGCohomology(BGG)
    cohom_dic = extend_from_symmetry(cohom_dic)
    latex_dic = {k:cohom.cohom_to_latex(c, compact=compact) for k,c in cohom_dic.items()}
    betti_dic = {k:cohom.betti_number(c) for k,c in cohom_dic.items()}
    tab1 = display_bigraded_table(latex_dic)
    tab2 = display_bigraded_table(betti_dic)
    tab3 = display_cohomology_stats(cohom_dic, BGG)
    with open(texfile, 'w') as f:
        f.write(prepare_texfile([tab1,tab2,tab3],title=f'type {diagram}, s={s}, subset={subset}'))

  0%|          | 0/36 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

--------------------------------------------------
type A4, table for s=0:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

 
*** Profile stats marshalled to file 'prun.log'.


         1518172817 function calls (1514186745 primitive calls) in 2506.831 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       30 1112.115   37.071 1684.716   56.157 quantum_center.py:166(Eijk_basis)
       30  174.340    5.811  174.501    5.817 quantum_center.py:342(_quotient_basis_indices)
    32658  137.402    0.004  141.377    0.004 {method 'solve_right' of 'sage.matrix.matrix2.Matrix' objects}
      546  103.046    0.189  311.005    0.570 quantum_center.py:426(_compute_kernel)
       90  100.875    1.121  310.869    3.454 la_modules.py:97(__init__)
     3566   99.010    0.028  141.792    0.040 cohomology.pyx:325(coker_reduce)
    29335   81.924    0.003  136.072    0.005 cohomology.pyx:75(merge_sorted_image)
    17019   63.248    0.004   64.505    0.004 cohomology.pyx:20(compute_action)
      180   55.855    0.310  207.757    1.154 la_modules.py:249(compute_weight_components)
 41680619   47.643    0.000   47.643   

In [5]:
import pstats
import io

result = io.StringIO()
stats = pstats.Stats('prun.log',stream=result)
stats.sort_stats("cumulative").print_stats()
result=result.getvalue()
# chop the string into a csv-like buffer
result='ncalls'+result.split('ncalls')[-1]
result='\n'.join([','.join(line.rstrip().split(None,5)) for line in result.split('\n')])
# save it to disk
 
with open('profile.csv', 'w+') as f:
    #f=open(result.rsplit('.')[0]+'.csv','w')
    f.write(result)
    f.close()